In [ ]:
# import the necessary packages

import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import seaborn as sns


# DBSCAN algorithm
We perform the density-based clustering using DBSCAN (Density-Based Spatial Clustering of Applications with Noise) algorithm.
It has only two parameters...

- eps: The maximum distance between two samples for them to be considered as in the same neighborhood.
- min_samples: the minimum number of points needed to create a cluster.
... and one output:
- cluster labels for each point in the dataset. Noisy samples are given the label -1.

## Pre-processing
We will use the normalized dataset with Min-Max approach to perform DBSCAN.

In [ ]:
df = pd.read_csv('../data/extracted_features.csv')

standard = StandardScaler()
minmax = MinMaxScaler()

#pick the California data
df = df[df['state'] == 'California']
df = df.dropna(subset=['avg_age_participants', 'n_involved', 'n_minors', 'kills_to_pov' ,'category_index'])

# plot correlation matrix to see if there is any feature to drop
plt.figure(figsize=(10, 8))
correlation_matrix = df.select_dtypes(include='number').corr()
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt='.2f', linewidths=.5, annot_kws={"size": 10})

In [ ]:
df[['avg_age_participants', 'n_involved', 'n_minors', 'category_index','kills_to_pov']].info()

In [ ]:

new_df = df[['avg_age_participants', 'n_involved', 'n_minors', 'category_index', 'kills_to_pov']]



# remove highly correlated features

# scale the data
ss_scaled = standard.fit_transform(new_df)
mm_scaled = minmax.fit_transform(new_df)

In [ ]:
from scipy.spatial.distance import pdist, squareform
import tqdm

def calc_k_nn_dist_graph(X, ub=0., lb=0.):
    # List of k (for k-th nearest neighbour)
    k_list = [5, 10, 15]

    dist = pdist(X, 'euclidean')  # Pairwise distance
    dist = squareform(dist)  # Distance matrix given the vector dist
    
    # Calculate sorted list of distances for points for each k
    for k in k_list:
        kth_distances = []
        for d in dist:
            index_kth_distance = np.argsort(d)[k]
            kth_distances.append(d[index_kth_distance])
 
        # Plot the graph of distance from k-th nearest neighbour
        plt.figure(figsize=(8, 6))
        plt.plot(range(0, len(kth_distances)), sorted(kth_distances))
        plt.ylabel('dist from %sth neighbor' % k, fontsize=18)
        plt.xlabel('sorted distances', fontsize=18)
        plt.tick_params(axis='both', which='major', labelsize=12)
        plt.grid()
        plt.axhline(ub)
        plt.axhline(lb)
        plt.show()
        
def db_scan_function(X, eps_list, minpts_list):
    # Table with dim = len(eps_list) x len(minpts_list)
    clustering_results = []

    # Total number of combinations
    total_combinations = len(eps_list) * len(minpts_list)
    current_combination = 0

    # Iterate over eps (rows)
    for eps in eps_list:
        minpts_results = []
        
        # Iterate over minpts
        for minpts in minpts_list:
            dbscan = DBSCAN(eps=eps, min_samples=minpts, n_jobs=-1)
            dbscan.fit(X)  # Fit the DBSCAN model to the data
            minpts_results.append(dbscan)  # Store the DBSCAN model
            
            # Update progress
            current_combination += 1
            print(f"Progress: {current_combination}/{total_combinations}")

        clustering_results.append(minpts_results)

    return clustering_results    

def print_dbscan(dbscan_table, eps_values, min_samples, dataframe):
    data = []
    columns = ['eps\\minpts'] + [str(minpt) for minpt in min_samples]

    for i, eps in enumerate(eps_values):
        row_data = [str(eps)]
        for j, minpts in enumerate(min_samples):
            dbscan = dbscan_table[i][j]
    
            try:  # Exception in case of num_clusters=1
                silhouette = round(silhouette_score(dataframe, dbscan.labels_), 2)
            except:
                silhouette = np.nan
            cell_str = str(silhouette) + '-n_clust:' + str(len(np.unique(dbscan.labels_)) - 1)
            row_data.append(cell_str)

        data.append(row_data)

    return pd.DataFrame(data, columns=columns)


In [ ]:
# values of min_samples to try
min_samples = range(5, 30, 5)

In [ ]:

# values of eps to try
eps_values = np.linspace(0.05, 0.5, 20)

#repeat the same for standard scaled data
calc_k_nn_dist_graph(ss_scaled, 0.1, 0.5)

dbscan_table = db_scan_function(ss_scaled, eps_values, min_samples)
scalar_result = print_dbscan(dbscan_table, eps_values, min_samples, ss_scaled)

In [ ]:
# values of eps to try
eps_values = np.linspace(0.005, 0.15, 10)

In [ ]:
calc_k_nn_dist_graph(mm_scaled, 0.01, 0.1)

In [ ]:
dbscan_table = db_scan_function(mm_scaled, eps_values, min_samples)
minmax_result = print_dbscan(dbscan_table, eps_values, min_samples, mm_scaled)

In [ ]:
scalar_result

In [ ]:
minmax_result

In [ ]:
# test the best parameters for the DBSCAN algorithm
silhouette_scores = []
parameter_combinations = []

best_score = -1  # Initialize best silhouette score
best_params = {'eps': None, 'min_samples': None}
for eps in eps_values:
    for min_sample in min_samples:
        dbscan = DBSCAN(eps=eps, min_samples=min_sample)
        labels = dbscan.fit_predict(mm_scaled)

        unique_labels = np.unique(labels)
        if len(unique_labels) < 3 or len(unique_labels) > 40: continue
        silhouette = silhouette_score(mm_scaled, labels)
        print(
            f"Number of clusters: {len(unique_labels)}, eps: {eps}, min_samples: {min_sample} , Silhouette score: {silhouette}")
        silhouette_scores.append(silhouette)
        parameter_combinations.append((eps, min_sample))

# Plotting the results
silhouette_scores = np.array(silhouette_scores)
parameter_combinations = np.array(parameter_combinations)


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(parameter_combinations[:, 0], parameter_combinations[:, 1], c=silhouette_scores, cmap='viridis', s=100)
plt.colorbar(label='Silhouette Score')
plt.xlabel('Epsilon (eps)')
plt.ylabel('Min Samples')
plt.title('Silhouette Scores for Different Parameter Combinations')
plt.grid(True)
plt.show()

eps_values = parameter_combinations[:, 0]
min_samples_values = parameter_combinations[:, 1]

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(eps_values, min_samples_values, silhouette_scores, c=silhouette_scores, cmap='viridis')
ax.set_xlabel('Eps Values')
ax.set_ylabel('Min Samples')
ax.set_zlabel('Silhouette Score')
ax.set_title('Silhouette Scores for DBSCAN Parameter Combinations')

plt.show()

In [ ]:
best_score_index = np.argmax(silhouette_scores)  # Get the index of the maximum silhouette score
best_params = parameter_combinations[best_score_index]  # Retrieve parameters corresponding to the best score

best_eps, best_min_samples = best_params  # Unpack the best parameters

best_score = silhouette_scores[best_score_index]  # Get the best silhouette score

# Print the best silhouette score and corresponding parameters
print(f"Best Silhouette Score: {best_score}")
print(f"Best Parameters - eps: {best_eps}, min_samples: {best_min_samples}")

In [ ]:
dbscan = DBSCAN(eps=best_eps, min_samples=int(best_min_samples))
cluster_labels = dbscan.fit_predict(mm_scaled)
selected_data = new_df.copy()
selected_data['cluster_labels'] = cluster_labels
sns.pairplot(selected_data, hue='cluster_labels', palette='viridis')
plt.suptitle('Pairplot of Features with Cluster Labels')
plt.show()

In [ ]:
if False:
    dbscan = DBSCAN(eps=0.3, min_samples=15)
    cluster_labels = dbscan.fit_predict(ss_scaled)
    selected_data = new_df.copy()
    selected_data['cluster_labels'] = cluster_labels
    sns.pairplot(selected_data, hue='cluster_labels', palette='viridis')
    plt.suptitle('Pairplot of Features with Cluster Labels')
    plt.show()

In [ ]:
import plotly.express as px

#remove data with -1 label
selected_data = selected_data[selected_data['cluster_labels'] != -1]

selected_data.head()
fig = px.scatter_mapbox(
    pd.merge(selected_data, df, how="right"), lat='latitude', lon='longitude',
    color='cluster_labels', mapbox_style="carto-positron",
    zoom=3, width=1000, height=600
)
fig.show()